In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.optim as optim

class TimeSeriesDataset(Dataset):
    """Dataset for loading and sliding over time-series data."""
    def __init__(self, topo_array, debris_array):
        """
        Initialize the dataset with topography and debris arrays.

        Args:
        topo_array (numpy.ndarray): The topography 2D array.
        debris_array (numpy.ndarray): The debris velocity or thickness 2D array.
        """
        # Ensure that the input arrays are of type float32
        self.topo_array = topo_array.astype('float32')
        self.debris_array = debris_array.astype('float32')
        # Combine the arrays along the last axis to create 3D dataset (time, height, width)
        self.data = np.stack((self.topo_array, self.debris_array), axis=-1)

    def __len__(self):
        """Return the number of timesteps in the dataset."""
        return self.data.shape[0] - 1  # minus 1 because we need pairs (input, next)

    def __getitem__(self, idx):
        """Get the (input, target) pair corresponding to the timestep idx."""
        return (self.data[idx], self.data[idx + 1])

def create_dataloaders(topo_array, debris_array, batch_size=32, test_size=0.2, random_state=42):
    """
    Create DataLoader instances for training and testing.

    Args:
    topo_array (numpy.ndarray): The topography 2D array.
    debris_array (numpy.ndarray): The debris velocity or thickness 2D array.
    batch_size (int): The batch size for DataLoader.
    test_size (float): The proportion of the dataset to include in the test split.
    random_state (int): The random state for reproducibility.

    Returns:
    DataLoader, DataLoader: The training and testing dataloaders.
    """
    dataset = TimeSeriesDataset(topo_array, debris_array)
    train_indices, test_indices = train_test_split(
        range(len(dataset)),
        test_size=test_size,
        random_state=random_state
    )
    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    test_dataset = torch.utils.data.Subset(dataset, test_indices)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader




def train_model(model, train_loader, test_loader, epochs=5, learning_rate=1e-3):
    """
    Train the neural network model.

    Args:
    model (nn.Module): The neural network model to train.
    train_loader (DataLoader): DataLoader for the training data.
    test_loader (DataLoader): DataLoader for the testing data.
    epochs (int): The number of epochs to train for.
    learning_rate (float): The learning rate for the optimizer.
    """
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Move the model to the GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for inputs, targets in train_loader:
            # Move data to the device
            inputs, targets = inputs.to(device), targets.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print statistics
        print(f'Epoch {epoch+1}/{epochs} - Loss: {running_loss / len(train_loader)}')

        # Evaluate the model on the test set
        model.eval()  # Set the model to evaluation mode
        test_loss = 0.0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()

        print(f'Test Loss: {test_loss / len(test_loader)}')

# Example usage:
# Assuming topo_array and debris_array are your preprocessed NumPy arrays
input_shape = (2, topo_array.shape[1], topo_array.shape[2])  # (channels, height, width)
model = SimpleNN(input_shape=input_shape)
train_loader, test_loader = create_dataloaders(topo_array, debris_array)
train_model(model, train_loader, test_loader)